In [ ]:
import numpy as np
import pandas as pd
from multiprocess import Pool

from benchmarking.multiprocessing.functions import evaluate_ar, evaluate_multiplier_iid, evaluate_ma
from bstrapping.synthetic_time_series.moving_average import MovingAverage

In [ ]:
mean = 4  # mean of the time series
alpha = 0.05

# Names of the stochastic processes

# Dependence coefficients of the stochastic processes, i.e. of the moving average processes
dependence_coefficients = [
    np.array([0]),
    np.array([0.5]),
    np.array([0.5 ** i for i in range(1, 3)]),
    np.array([0.5 ** i for i in range(1, 11)]),
    np.array([0.5 ** i for i in range(1, 16)]),
    np.array([0.5 ** i for i in range(1, 21)]),
]

names_dependence_coefficients = [
    "iid",
    "MA(1)",
    "MA(2)",
    "MA(10)",
    "MA(25)",
    "MA(20)",
]

list_name_weights = ['AR',
                     'Multiplier',
                     'MA',
                     ]

In [ ]:
benchmark = []
means_of_variance = []
std_of_variance = []
coverage_probability = []

In [ ]:
sample_size = 1000
runs = 100
index_dependence = 3

time_series = MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence])

In [ ]:
samples = [
    MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence]).generate_samples(sample_size)
    for _ in range(runs)]

In [ ]:
MovingAverage(mean=mean, parameters=dependence_coefficients[index_dependence]).asymptotic_variance

## Benchmark bootstraps

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_ar = p.map(lambda sample: evaluate_ar(sample, alpha=alpha), samples)

result = np.array(evaluations_ar)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))


In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_multiplier_iid = p.map(lambda sample: evaluate_multiplier_iid(sample, alpha=alpha), samples)

result = np.array(evaluations_multiplier_iid)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_ma = p.map(lambda sample: evaluate_ma(sample, alpha=alpha), samples)

result = np.array(evaluations_ma)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))

## Concatination result

In [ ]:
benchmark = [names_dependence_coefficients[index_dependence], time_series.asymptotic_variance,
             sample_size] + means_of_variance + std_of_variance + [1 - alpha] + coverage_probability

In [ ]:
benchmark = pd.DataFrame([benchmark], columns=pd.MultiIndex.from_tuples([("Stochastic process", ""),
                                                             ("mean", "Asymptotic variance"),
                                                             ("Sample size", "")] +
                                                            [("mean", name,) for name in list_name_weights] +
                                                            [("std", name,) for name in list_name_weights] +
                                                            [("In confidence interval", "Confidence level")]
                                                            +
                                                            [("In confidence interval", name,) for name in
                                                             list_name_weights]
                                                            )).set_index(["Sample size"])

In [ ]:
benchmark.to_csv(f"./data/benchmark_{sample_size}_{names_dependence_coefficients[index_dependence]}.csv")
benchmark.to_pickle(f"./data/benchmark_{sample_size}_{names_dependence_coefficients[index_dependence]}.pkl")

In [ ]:
benchmark